# Lecture 7 (Week 10) Exercise / Homework #3

For this assignment, you will learn about two new libraries, pyttsx3 and speech_recognition, and use voice recognition to obtain weather info <br />

**pyttsx3** - https://pypi.org/project/pyttsx3/ <br />
**speech_recognition** - https://pypi.org/project/SpeechRecognition/ <br />

**pyAudio** - PyAudio‑0.2.11‑cp38‑cp38‑win_amd64.whl <br />
Use following stackoverflow post to troubleshoot if needed 
https://stackoverflow.com/questions/61290821/error-command-errored-out-with-exit-status-1-while-installing-pyaudio

First, try to import the two libraries to see if can import the modules, if not, you need to do pip install to have them installed through terminal (MAC) or command line (MS)

In [1]:
import pyttsx3
import speech_recognition as sr
import smtplib
from os.path import basename
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.mime.text import MIMEText
import yfinance as yf
from pandas_datareader import data as pdr
import pandas as pd
from yahoo_fin import stock_info as si

Try out the test script from ptytsx3 website, see if it runs, did you hear the computer speak?

In [2]:
engine = pyttsx3.init()
engine.say("I will speak this text")
engine.runAndWait()

Based on website description, see if you can make engine speak in female voice (**hint:** try voices 1, and 10)

In [3]:
voices = engine.getProperty('voices') 
engine.setProperty('voice', voices[1].id)  
engine.say("I will speak this text")
engine.runAndWait()

Check your microphone

In [4]:
for index, name in enumerate(sr.Microphone.list_microphone_names()):
    print(f'{index}, {name}')

0, Microsoft Sound Mapper - Input
1, Microphone Array (Intel® Smart 
2, Microsoft Sound Mapper - Output
3, Speakers (Realtek(R) Audio)
4, Primary Sound Capture Driver
5, Microphone Array (Intel® Smart Sound Technology (Intel® SST))
6, Primary Sound Driver
7, Speakers (Realtek(R) Audio)
8, Speakers (Realtek(R) Audio)
9, Microphone Array (Intel® Smart Sound Technology (Intel® SST))
10, Headphones 1 (Realtek HD Audio 2nd output with SST)
11, Headphones 2 (Realtek HD Audio 2nd output with SST)
12, PC Speaker (Realtek HD Audio 2nd output with SST)
13, Stereo Mix (Realtek HD Audio Stereo input)
14, Speakers 1 (Realtek HD Audio output with SST)
15, Speakers 2 (Realtek HD Audio output with SST)
16, PC Speaker (Realtek HD Audio output with SST)
17, Microphone (Realtek HD Audio Mic input)
18, Headset (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free%0
;(Soundcore Life Q30))
19, Headset (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free%0
;(Soundcore Life Q30))
20, Headphones ()
21, Headphones ()


Now let's try listening, try to speak to it, does it provide correct return?
**Note:** only talk to the computer after "listening" is printed

In [5]:
##Make sure you install pyaudio first!!

listener = sr.Recognizer()

try:
    with sr.Microphone(device_index = 1) as source:
        print('listening...')
        voice = listener.listen(source, timeout=4) ##timeout after 2 seconds
        command = listener.recognize_google(voice, language = 'us-eng')
        print(command)
except:
    print("Nothing is Captured")

listening...
is international


Now you can get Python to both listen and talk, let's leverage what we learned in class and have computer tell us weather of a city
for the first test, let's only worry about the "happy path", don't worry about error for now
1. Computer initiate by asking what city you are looking for?
2. User provide a city
3. Computer ask for state abbreviation
4. User provide state abrreviation 
5. System reads out weather description, temperature and feels like (all part of json readout) 

In [6]:
from urllib.request import urlopen  ##import a function
import json

In [71]:
def listen():
    listener = sr.Recognizer()
    captured_text = ''
    try:
        with sr.Microphone(device_index = 0) as source:
            print('listening...')
            listener.adjust_for_ambient_noise(source, duration=0.5)
            voice = listener.listen(source, timeout=2) ##timeout after 6 seconds
            command = listener.recognize_google(voice, language = 'us-eng')
            print(command)
    except:
        print("Nothing is Captured")
    return command

def speak(sentence):
    engine = pyttsx3.init()
    engine.say(sentence)
    engine.runAndWait()

def weather(city, state):
    api = '09f706cb90f6cc9bf11e5e4bc021a030'
    city = '+'.join(city.lower().split())
    state = state.lower()
    link = f"http://api.openweathermap.org/data/2.5/weather?q={city},{state},US&appid={api}"
    response = urlopen(link)
    input1 = response.read().decode("utf-8") 
    final_input = json.loads(input1)
    description = final_input['weather'][0]['description']
    outputs = final_input['main']
    vals = []
    iteration=0
    for field in outputs:
        if iteration <= 2:
            C = outputs[field] - 273
            F = 1.8*C + 32
            val = round(F)
        else:
            val = outputs[field]
        vals.append(val)
        iteration += 1
    description_readout = "We expect {}.\n".format(description)
    temp_readout = """
    actual: {},
    feels like: {}.
    The low temperature for today is {} and the high temperature is {}.
    """.format(*vals)
    
    final_output = description_readout + temp_readout
    return final_output

def final():
    city = ''
    state = ''
    i = 1
    speak("What city would you like to know the weather for?")
    city = listen()
    speak(f"What is the two letter state abbreviation of {city}?")
    state = listen()
    report = weather(city=city, state=state)
    print(report)
    speak(report)
    
    

In [72]:
final()

listening...
Los Angeles
listening...
CA
We expect clear sky.

    actual: 58,
    feels like: 58.
    The low temperature for today is 55 and the high temperature is 62.
    


Were there times the voice didn't capture correctly? Convert it into a function
Please leverage various parameters with speech_recognition to optimize output
In addition, please write some looping code to loop back for potential errors
1. For example, if 'MJ' is captured instead of 'NJ', we should ask user to confirm
2. Timeout errors

Let's leverage what you learned in class this week and design a quick tool

In [7]:
import pandas as pd
import numpy as np

###Symbol column will provide you ticker of all 30 index
ticker = pd.read_html('https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average')
ticker[1]

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1123)>

In [8]:
###Pull volume of stock for individual ticker
import requests
url_link = 'https://finance.yahoo.com/quote/MMM/key-statistics?p=MMM'
enhanced_link = requests.get(url_link,headers ={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'})
MMM = pd.read_html(enhanced_link.text)
MMM[0]

,Unnamed: 0,As of Date: 3/30/2022Current,12/31/2021,9/30/2021,6/30/2021,3/31/2021
0,Market Cap (intraday),86.56B,101.58B,101.09B,114.93B,111.69B
1,Enterprise Value,100.10B,114.85B,114.71B,128.86B,126.41B
2,Trailing P/E,15.03,17.57,17.35,20.33,20.83
3,Forward P/E,14.62,17.15,16.29,20.49,19.96
4,PEG Ratio (5 yr expected),2.09,2.16,1.82,2.13,3.25
5,Price/Sales (ttm),2.52,2.95,2.96,3.52,3.49
6,Price/Book (mrq),5.75,6.99,6.96,8.35,8.68
7,Enterprise Value/Revenue,2.83,13.34,12.83,14.40,14.28
8,Enterprise Value/EBITDA,10.42,52.47,48.81,50.91,49.83


Write a program that will send your self an email on stock summary based on last trading day (You may assume the code is only ran after market is closed)
1. Stock Ticker with highest volume
2. Stock Ticker with largest fluctuation (High - Low) / Low
3. Stock Ticker with max gain (Close-Open) / Open
4. Stock Ticket with max loss or min gain (Close-Open) / Open
Once you have your code finalized, have a copy send to rutgersadvancedpython01@outlook.com

In [9]:
resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
table1 = pd.read_html(resp.text)
tickers = list(table1[0]['Symbol'])
tickers

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADM',
 'ADBE',
 'ADP',
 'AAP',
 'AES',
 'AFL',
 'A',
 'AIG',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AMD',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ANET',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BLL',
 'BAC',
 'BBWI',
 'BAX',
 'BDX',
 'WRB',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'CHRW',
 'CDNS',
 'CZR',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CDAY',
 'CERN',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMC

In [10]:
check = pd.DataFrame()

for i in tickers:
    tickerSymbol = i

    tickerData = yf.Ticker(tickerSymbol)

    tickerDf = tickerData.history(period='1d', start='2022-4-1', end='2022-4-2')
    tickerDf['stock'] = tickerSymbol
    tickerDf.set_index('stock', inplace = True)

#see your data
    check = check.append(tickerDf)



- BRK.B: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted


In [11]:
check.reset_index(inplace =True)

In [44]:
check['fluctuation'] =(check['High'] - check['Low']) / check['Low']
check['gain']=(check['Close']-check['Open']) / check['Open']

fluctuation_stock =check[check['fluctuation']==check['fluctuation'].max()]['stock'].item()
max_stock =check[check['gain']==check['gain'].max()]['stock'].item()
min_stock =check[check['gain']==check['gain'].min()]['stock'].item()

max_stock

'NEM'

In [50]:
text = 'The stock which showed the highest fluctuation is ' + fluctuation_stock + \
         ',The stock which showed the highest gain is  ' + max_stock + \
        ',and The stock which showed the least gain is  ' + min_stock 

In [51]:
msg = MIMEMultipart()

msg['Subject'] = "Stock news"
msg['From'] = 'nakul.datla@gmail.com'
msg['To'] = 'rajukrishnam1997@gmail.com'

###insert password
f = open("pass.txt", "r")
password = f.read()

html = """\
<html>
  <head></head>
    <body>
       <p>
       """ + text + """\
       </p>   
       <img src="cid:image1" alt="Logo"><br>
    </body>
</html>
"""
# Record the MIME types of text/html.
part2 = MIMEText(html, 'html')

# Attach parts into message container.
msg.attach(part2)


s = smtplib.SMTP(host='smtp.gmail.com', port=587)
s.ehlo()
s.starttls()
s.login('nakul.datla@gmail.com', password)
s.sendmail('nakul.datla@gmail.com', 'rutgersadvancedpython01@outlook.com', msg.as_string())

{}